In [1]:
import pandas as pd

In [4]:
dataset=pd.read_csv("Attrition_Rate_Analysis.csv")

In [6]:
dataset.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EmployeeCount', 'EmployeeID', 'Gender',
       'JobLevel', 'JobRole', 'MaritalStatus', 'MonthlyIncome',
       'NumCompaniesWorked', 'Over18', 'PercentSalaryHike', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')

In [7]:
#convert DV,IDV variable from text to numeric

In [10]:
from sklearn import preprocessing

In [11]:
label_encoder=preprocessing.LabelEncoder()

In [12]:
dataset["Attrition"]=label_encoder.fit_transform(dataset["Attrition"])#transform converts text inti numeric

In [13]:
dataset["BusinessTravel"]=label_encoder.fit_transform(dataset["BusinessTravel"])#transform converts text inti numeric

In [14]:
dataset["Department"]=label_encoder.fit_transform(dataset["Department"])#transform converts text inti numeric

In [17]:
dataset["EducationField"]=label_encoder.fit_transform(dataset["EducationField"])#transform converts text inti numeric

In [18]:
dataset["Gender"]=label_encoder.fit_transform(dataset["Gender"])#transform converts text inti numeric

In [19]:
dataset["MaritalStatus"]=label_encoder.fit_transform(dataset["MaritalStatus"])#transform converts text inti numeric

In [20]:
dataset["JobRole"]=label_encoder.fit_transform(dataset["JobRole"])#transform converts text inti numeric

In [21]:
# remove coulumns not required

In [22]:
dataset1=dataset.drop(["EmployeeCount","EmployeeID","Over18","StandardHours"],axis=1)

In [25]:
dataset1.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole',
       'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
       'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [26]:
#also verify whether if they have any null value to drop or remove null values

In [27]:
dataset2=dataset1.dropna()

In [28]:
dataset3=dataset2.drop_duplicates()

In [29]:
# initiate a Random forest Alog because in this case we have mainy independent variable so purning is required

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
import numpy as np

In [32]:
rf_model=RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)#n_estimators are iteration,max_features is birany split

In [34]:
features=['Age','BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole',
       'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
       'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
       'YearsWithCurrManager']

In [35]:
rf_model.fit(X=dataset3[features],y=dataset3["Attrition"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [36]:
print("OOB Accuracy:")
print(rf_model.oob_score_)

OOB Accuracy:
0.8448979591836735


In [37]:
# To find the importance of each variable

In [38]:
for feature, imp in zip(features,rf_model.feature_importances_):
    print(feature,imp)
    

Age 0.09544623981000666
BusinessTravel 0.027254247626609104
Department 0.026436584728122853
DistanceFromHome 0.06997442820672164
Education 0.04069054330898054
EducationField 0.04162359998732411
Gender 0.018517480549893092
JobLevel 0.03703530289239162
JobRole 0.05544345232870895
MaritalStatus 0.04011388534897771
MonthlyIncome 0.09430504499693912
NumCompaniesWorked 0.05574446337735886
PercentSalaryHike 0.06561477085982631
StockOptionLevel 0.03415688699819209
TotalWorkingYears 0.08642546243586492
TrainingTimesLastYear 0.0440280169260215
YearsAtCompany 0.06883014401747899
YearsSinceLastPromotion 0.04384733797540441
YearsWithCurrManager 0.05451210762517759


In [39]:
#Select variable which have highest importance or value Age,MonthlyIncome ,TotalWorkingYears 

In [40]:
# Lets draw the decision Tree for the important variable

In [41]:
from sklearn import tree

In [42]:
tree_model=tree.DecisionTreeClassifier()

In [43]:
tree_model=tree.DecisionTreeClassifier(max_depth=6,max_leaf_nodes=10)# as we have 3 imp variable and DV hav Y and N so depth is 3*2=6

In [46]:
predictors=pd.DataFrame([dataset3["Age"],dataset3["MonthlyIncome"],dataset3["TotalWorkingYears"]]).T

In [47]:
tree_model.fit(X=predictors,y=dataset3["Attrition"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [48]:
with open("Dtree2.dot",'w') as f:
    f=tree.export_graphviz(tree_model,feature_names=["Age","MonthlyIncome","TotalWorkingYears"],out_file=f);